In [3]:
import keras as K
import os.path

import sys

sys.path.append("../src")
from artstat import util
import numpy as np

Using TensorFlow backend.


In [4]:
import imp
imp.reload(util)

<module 'artstat.util' from '../src/artstat/util.py'>

In [5]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"

In [159]:
words, vocab = util.load_vocab("../vocab.txt", 100000)

In [160]:
emb_matrix = util.load_embeddings(vocab, 300, glove)

In [161]:
emb_matrix.shape
len(vocab)

100000

In [162]:
imp.reload(util)

seqlen = 128
stride = 96
X, Y, Xu, Yu = util.load_data_sequences(path_train, vocab, seqlen, stride, numfiles=100)
Yfake = np.ones((Yu.shape[0], 128), dtype="float32")


  0%|          | 54/22860 [00:00<00:42, 539.23it/s]


In [163]:
input_x = K.Input((seqlen,), dtype="int32", name="input_x")
input_xu = K.Input((seqlen,), dtype="float32", name="input_xu")
input_y = K.Input((seqlen,), dtype="int32", name="input_y")
input_yu = K.Input((seqlen,), dtype="float32", name="input_yu")

reshaper_u  = K.layers.Reshape((seqlen,1))
resh_xu = reshaper_u(input_xu)
resh_yu = reshaper_u(input_yu)

dim = emb_matrix.shape[1] + 1

L_emb = K.layers.Embedding(emb_matrix.shape[0], 
                           emb_matrix.shape[1], 
                           input_length=seqlen, 
                          trainable=False,
                           weights=[emb_matrix],
                           name="embedding")


In [164]:
Lemb = L_emb
emb_x = Lemb(input_x)
emb_y = Lemb(input_y)

In [165]:
Lconcat = K.layers.Concatenate()
concat_x = Lconcat([emb_x, resh_xu])
concat_y = Lconcat([emb_y, resh_yu])

In [166]:
lstm =  K.layers.CuDNNLSTM(256, return_sequences=True)(concat_x)
lstm =  K.layers.CuDNNLSTM(256, return_sequences=True)(concat_x)
#lstm = K.layers.CuDNNLSTM(256, return_sequences=True)(lstm)
dense = K.layers.BatchNormalization()(lstm)
dense = K.layers.Dense(1024, activation="relu")(dense)
dense = K.layers.BatchNormalization()(dense)
dense = K.layers.Dense(1024, activation="relu")(dense)
dense = K.layers.BatchNormalization()(dense)
dense = K.layers.Dense(dim, activation="sigmoid")(dense)

dense.shape

TensorShape([Dimension(None), Dimension(128), Dimension(301)])

In [167]:
concat_y.shape

TensorShape([Dimension(None), Dimension(128), Dimension(301)])

In [168]:
#proximity = K.layers.dot([resh_x, resh_y], axes=-1, normalize=True)
import keras as K
from keras import backend as B

import tensorflow as tf


epsi = 0.0001

def realdot(a,b):
    d = K.layers.multiply([a, b])
    return B.sum(d, axis=-1)

def dotlayer(x):
    a,b = x
    
    p = realdot(a,b)
    
    an = B.sqrt(realdot(a,a) + B.epsilon())
    bn = B.sqrt(realdot(b,b) + B.epsilon())
    
    return tf.divide(p, K.layers.multiply([an, bn]) + B.epsilon())
    
def test2(x):
    a,b = x
    
    q = K.layers.subtract([a,b])
    q = B.sum(q, axis=-1)
#    q = B.mean(q, axis=-1, keepdims=True)
    return q

qA = B.variable(np.random.randn(6,10,12)) #np.array([[1,0], [3,4]]))
qB = B.variable(np.random.randn(6,10,12)) #np.array([[0,0], [-7,-8]]))
qq = dotlayer([qA, qB])
print(qq)
print(B.eval(qq))    
    
proximity = K.layers.Lambda(dotlayer)([concat_y, dense])
    
#proximity = K.layers.dot([concat_y, dense], axes=(2,2),normalize=True)




#proximity = K.layers.average(proximity, axis=-1)
proximity.shape

Tensor("truediv_10:0", shape=(6, 10), dtype=float32)
[[ 0.21235225 -0.24089421  0.31849968 -0.06436007  0.17942332  0.07761139
  -0.03735921 -0.4905576  -0.03229325  0.26053354]
 [ 0.20631321  0.23059595  0.544917    0.05250033 -0.25267574  0.20119864
  -0.46314734 -0.22291759 -0.2247903   0.10972435]
 [-0.19505201 -0.14896832 -0.23880954 -0.11091805 -0.27862975  0.18430588
   0.07154792  0.11619413  0.00159042 -0.05508105]
 [ 0.05807852  0.01318557 -0.03135784 -0.07669993  0.12554404  0.3934909
   0.37591565  0.22530068  0.69247246 -0.05288764]
 [-0.2564594  -0.17248338  0.1345384   0.0564288   0.04351333 -0.13689366
  -0.00105489 -0.13824148  0.07972438 -0.0160603 ]
 [ 0.02176644  0.25392687  0.14088386 -0.02723447  0.12549354 -0.09937479
   0.62498444 -0.31078127  0.47853023 -0.45502126]]


TensorShape([Dimension(None), Dimension(128)])

In [169]:
model = K.Model(inputs=[input_x, input_xu, input_y, input_yu],
               outputs=[proximity])

In [170]:
i = 500
z = model.predict([X[:i], Xu[:i], Y[:i], Yu[:i]])


In [171]:
loss = K.losses.mean_squared_error(Yfake[:i], z)

L = K.backend.eval(loss)

In [172]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_y (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_yu (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
input_x (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_xu (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
embedding 

In [173]:
g = B.gradients(L, L_dense.get_weights())

g


[None, None]

In [174]:
opt = K.optimizers.Adam(lr=0.01)
model.compile(opt, loss='mean_squared_error')

In [175]:
model.fit([X, Xu, Y, Yu], [Yfake], batch_size=1024, epochs=1000000)


Epoch 1/1000000
617/617 [==============================] - 2s 3ms/step - loss: 0.9712
Epoch 2/1000000
617/617 [==============================] - 0s 501us/step - loss: 0.8673
Epoch 3/1000000
617/617 [==============================] - 0s 509us/step - loss: 0.8007
Epoch 4/1000000
617/617 [==============================] - 0s 501us/step - loss: 0.7639
Epoch 5/1000000
617/617 [==============================] - 0s 480us/step - loss: 0.7206
Epoch 6/1000000
617/617 [==============================] - 0s 551us/step - loss: 0.6765
Epoch 7/1000000
617/617 [==============================] - 0s 555us/step - loss: 0.6311
Epoch 8/1000000
617/617 [==============================] - 0s 532us/step - loss: 0.5783
Epoch 9/1000000
617/617 [==============================] - 0s 495us/step - loss: 0.5386
Epoch 10/1000000
617/617 [==============================] - 0s 516us/step - loss: 0.5060
Epoch 11/1000000
617/617 [==============================] - 0s 595us/step - loss: 0.4843
Epoch 12/1000000
617/617 [======

617/617 [==============================] - 0s 515us/step - loss: 0.3689
Epoch 94/1000000
617/617 [==============================] - 0s 595us/step - loss: 0.3698
Epoch 95/1000000
617/617 [==============================] - 0s 490us/step - loss: 0.3731
Epoch 96/1000000
617/617 [==============================] - 0s 512us/step - loss: 0.3699
Epoch 97/1000000
617/617 [==============================] - 0s 542us/step - loss: 0.3690
Epoch 98/1000000
617/617 [==============================] - 0s 529us/step - loss: 0.3672
Epoch 99/1000000
617/617 [==============================] - 0s 521us/step - loss: 0.3660
Epoch 100/1000000
617/617 [==============================] - 0s 505us/step - loss: 0.3653
Epoch 101/1000000
617/617 [==============================] - 0s 504us/step - loss: 0.3641
Epoch 102/1000000


KeyboardInterrupt: 

In [136]:
predictor = K.Model([input_x, input_xu], [dense])
predictor.compile(opt, loss="mean_squared_error")

In [137]:
from sklearn import preprocessing as pp
norm_emb_matrix =  pp.normalize(Lemb.get_weights()[0], axis=1)

In [143]:
i = 3
gen = X[i].tolist()
genu = Xu[i].tolist()

tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype="float32")
results = []

iterations = 128
for j in range(iterations):
    tX[0,:] = np.array(gen[-seqlen:], "int32")
    tXu[0,:] = np.array(genu[-seqlen:], "float32")
    #print(tX)
    xv, xuv = 0, 0.0
    z = predictor.predict([tX, tXu])
    if False: #z[0, -1, -1] > 0.9:
        xuv = 1.0
        results.append("<UNK>")
    else:
        we = np.transpose(pp.normalize(np.reshape(z[0, -1, :300], (1,300))))
#        print(np.linalg.norm(we))
#        scores = np.matmul(norm_emb_matrix, we)
#        print(scores)
#        print(np.min(scores))
#        break
        xv = np.argmax(np.matmul(norm_emb_matrix, we))
        if xv:
            results.append(words[xv])
        else:
            results.append("<EMPTY>")
    gen.append(xv)
    genu.append(xuv)
    print(results[-1])

as
the
be
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
begin
the
architectural
example
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as
the
as


In [66]:
print(np.min(z), np.max(z))

1.3547268e-16 0.93638355


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'